What is a Lexer?
A Lexer (short for Lexical Analyzer) is the first phase of a compiler. Its job is to read raw source code (as plain text) and convert it into a sequence of tokens that are meaningful for the next stage — the parser.

What is a Parser?
The parser calls the lexer to get tokens. It processes tokens according to grammar rules (for, declarations, expressions) and then builds a nested AST that can be used for further analysis or code generation. It also reports syntax errors.



In [13]:
import re

TOKEN_SPEC = [
    ('COMMENT_SINGLE', r'//.*'),          # Single-line comment
    ('ML_COMMENT_START', r'/\*'),         # Multi-line comment start
    ('STRING_START', r'"'),                # String literal start (double quotes)
    ('NUMBER',    r'\d+'),                
    ('ID',        r'[A-Za-z_]\w*'),      
    ('ASSIGN',    r'='),                  
    ('SEMI',      r';'),                  
    ('PUNC',      r'[,.]'),               
    ('LBRACS',    r'[\(\[\{]'),           
    ('RBRACS',    r'[\)\]\}]'),           
    ('OP',        r'[+\-*/<>=!]+'),       
    ('SKIP',      r'[ \t]+'),             
    ('NEWLINE',   r'\n'),                 
    ('MISMATCH',  r'.'),                  
]

token_re = re.compile('|'.join(f'(?P<{name}>{regex})' for name, regex in TOKEN_SPEC))

def lexer(code):
    line_num = 1
    line_start = 0
    pos = 0
    length = len(code)
    
    while pos < length:
        match = token_re.match(code, pos)
        if not match:
            raise SyntaxError(f'Unexpected character {code[pos]!r} at line {line_num}, col {pos - line_start + 1}')
        
        kind = match.lastgroup
        value = match.group()
        start = match.start()

        if kind == 'NEWLINE':
            line_num += 1
            line_start = match.end()
            pos = match.end()
            continue
        elif kind == 'SKIP':
            pos = match.end()
            continue
        elif kind == 'COMMENT_SINGLE':
            pos = match.end()
            yield kind, value, line_num, start - line_start + 1
            continue
        elif kind == 'ML_COMMENT_START':
            end_pos = code.find('*/', match.end())
            if end_pos == -1:
                col = start - line_start + 1
                raise SyntaxError(f'Unterminated multi-line comment at line {line_num}, col {col}')
            comment_text = code[pos:end_pos+2]
            newlines = comment_text.count('\n')
            if newlines > 0:
                line_num += newlines
                # Update line_start to position after last newline in comment
                last_newline_pos = comment_text.rfind('\n')
                line_start = pos + last_newline_pos + 1
            pos = end_pos + 2
            yield 'COMMENT_MULTI', comment_text, line_num, start - line_start + 1
            continue
        elif kind == 'STRING_START':
            # Find closing quote
            end_pos = pos + 1
            escaped = False
            while end_pos < length:
                c = code[end_pos]
                if c == '\\' and not escaped:
                    escaped = True
                elif c == '"' and not escaped:
                    break
                else:
                    escaped = False
                end_pos += 1
            else:
                col = start - line_start + 1
                raise SyntaxError(f'Unterminated string literal at line {line_num}, col {col}')
            string_text = code[pos:end_pos+1]
            # Count newlines inside string (rare but possible with \n escapes or multiline strings)
            newlines = string_text.count('\n')
            if newlines > 0:
                line_num += newlines
                last_newline_pos = string_text.rfind('\n')
                line_start = pos + last_newline_pos + 1
            pos = end_pos + 1
            yield 'STRING', string_text, line_num, start - line_start + 1
            continue
        elif kind == 'MISMATCH':
            col = start - line_start + 1
            raise SyntaxError(f'Unexpected character {value!r} at line {line_num}, col {col}')
        else:
            if kind == 'NUMBER':
                value = int(value)
            elif kind == 'ID' and value in ('int', 'if', 'else', 'for'):
                kind = 'KEYWORD'
            col = start - line_start + 1
            yield kind, value, line_num, col
        
        pos = match.end()

# Example test: unterminated string

code = '''
for (int i=0; i<100; i=i+1) { // This comment never ends
  a[i] = a[i+1] + 1;
}
'''

try:
    for token in lexer(code):
        print(token)
except SyntaxError as e:
    print("Lexer Error:", e)


('KEYWORD', 'for', 2, 1)
('LBRACS', '(', 2, 5)
('KEYWORD', 'int', 2, 6)
('ID', 'i', 2, 10)
('ASSIGN', '=', 2, 11)
('NUMBER', 0, 2, 12)
('SEMI', ';', 2, 13)
('ID', 'i', 2, 15)
('OP', '<', 2, 16)
('NUMBER', 100, 2, 17)
('SEMI', ';', 2, 20)
('ID', 'i', 2, 22)
('ASSIGN', '=', 2, 23)
('ID', 'i', 2, 24)
('OP', '+', 2, 25)
('NUMBER', 1, 2, 26)
('RBRACS', ')', 2, 27)
('LBRACS', '{', 2, 29)
('COMMENT_SINGLE', '// This comment never ends', 2, 31)
('ID', 'a', 3, 3)
('LBRACS', '[', 3, 4)
('ID', 'i', 3, 5)
('RBRACS', ']', 3, 6)
('ASSIGN', '=', 3, 8)
('ID', 'a', 3, 10)
('LBRACS', '[', 3, 11)
('ID', 'i', 3, 12)
('OP', '+', 3, 13)
('NUMBER', 1, 3, 14)
('RBRACS', ']', 3, 15)
('OP', '+', 3, 17)
('NUMBER', 1, 3, 19)
('SEMI', ';', 3, 20)
('RBRACS', '}', 4, 1)


In [14]:
class Parser:
    def __init__(self, tokens):
        self.tokens = [t for t in tokens if t[0] != 'COMMENT_SINGLE' and t[0] != 'COMMENT_MULTI']
        self.pos = 0

    def peek(self):
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        return ('EOF', None, None, None)

    def consume(self, expected_kind=None, expected_value=None):
        tok = self.peek()
        if expected_kind and tok[0] != expected_kind:
            raise SyntaxError(f"Expected {expected_kind} but got {tok[0]} at line {tok[2]}, col {tok[3]}")
        if expected_value and tok[1] != expected_value:
            raise SyntaxError(f"Expected {expected_value} but got {tok[1]} at line {tok[2]}, col {tok[3]}")
        self.pos += 1
        return tok

    def parse(self):
        stmts = []
        while self.peek()[0] != 'EOF':
            stmts.append(self.statement())
        return ('program', stmts)

    def statement(self):
        tok = self.peek()
        if tok[0] == 'KEYWORD':
            if tok[1] == 'for':
                return self.for_loop()
            elif tok[1] == 'if':
                return self.if_statement()
            elif tok[1] == 'while':
                return self.while_statement()
            elif tok[1] == 'return':
                return self.return_statement()
            elif tok[1] == 'int':
                return self.declaration()
        elif tok[0] == 'LBRACS' and tok[1] == '{':
            return self.block()
        else:
            return self.expr_statement()

    def for_loop(self):
        self.consume('KEYWORD', 'for')
        self.consume('LBRACS', '(')
        init = self.for_init()
        self.consume('SEMI')
        cond = self.expression()
        self.consume('SEMI')
        step = self.expression()
        self.consume('RBRACS', ')')
        body = self.statement()
        return ('for', init, cond, step, body)

    def for_init(self):
        tok = self.peek()
        if tok[0] == 'KEYWORD' and tok[1] == 'int':
            decls = []
            while True:
                decl = self.declaration_no_semi()
                decls.append(decl)
                if self.peek()[0] == 'PUNC' and self.peek()[1] == ',':
                    self.consume('PUNC', ',')
                else:
                    break
            return ('for_init_decls', decls)
        elif tok[0] == 'SEMI':
            return None
        else:
            expr = self.expression()
            return ('for_init_expr', expr)

    def declaration_no_semi(self):
        vartype = self.consume('KEYWORD')[1]
        varname = self.consume('ID')[1]
        expr = None
        if self.peek()[0] == 'ASSIGN':
            self.consume('ASSIGN')
            expr = self.expression()
        return ('declaration', vartype, varname, expr)

    def if_statement(self):
        self.consume('KEYWORD', 'if')
        self.consume('LBRACS', '(')
        cond = self.expression()
        self.consume('RBRACS', ')')
        then_branch = self.statement()
        else_branch = None
        if self.peek()[0] == 'KEYWORD' and self.peek()[1] == 'else':
            self.consume('KEYWORD', 'else')
            else_branch = self.statement()
        return ('if', cond, then_branch, else_branch)

    def while_statement(self):
        self.consume('KEYWORD', 'while')
        self.consume('LBRACS', '(')
        cond = self.expression()
        self.consume('RBRACS', ')')
        body = self.statement()
        return ('while', cond, body)

    def return_statement(self):
        self.consume('KEYWORD', 'return')
        expr = None
        if self.peek()[0] != 'SEMI':
            expr = self.expression()
        self.consume('SEMI')
        return ('return', expr)

    def declaration(self):
        vartype = self.consume('KEYWORD')[1]
        varname = self.consume('ID')[1]
        expr = None
        if self.peek()[0] == 'ASSIGN':
            self.consume('ASSIGN')
            expr = self.expression()
        self.consume('SEMI')
        return ('declaration', vartype, varname, expr)

    def block(self):
        self.consume('LBRACS', '{')
        stmts = []
        while not (self.peek()[0] == 'RBRACS' and self.peek()[1] == '}'):
            stmts.append(self.statement())
        self.consume('RBRACS', '}')
        return ('block', stmts)

    def expr_statement(self):
        expr = self.expression()
        self.consume('SEMI')
        return ('expr_stmt', expr)

    # Expressions with comparison operators

    def expression(self):
        return self.assignment()

    def assignment(self):
        node = self.comparison()
        if self.peek()[0] == 'ASSIGN':
            self.consume('ASSIGN')
            right = self.assignment()
            return ('assign', node, right)
        return node

    def comparison(self):
        node = self.add_sub()
        while True:
            tok = self.peek()
            if tok[0] == 'OP' and tok[1] in ('<', '>', '<=', '>=', '==', '!='):
                op = tok[1]
                self.consume('OP')
                right = self.add_sub()
                node = ('binop', op, node, right)
            else:
                break
        return node

    def add_sub(self):
        node = self.mul_div()
        while True:
            tok = self.peek()
            if tok[0] == 'OP' and tok[1] in ('+', '-'):
                op = tok[1]
                self.consume('OP')
                right = self.mul_div()
                node = ('binop', op, node, right)
            else:
                break
        return node

    def mul_div(self):
        node = self.unary()
        while True:
            tok = self.peek()
            if tok[0] == 'OP' and tok[1] in ('*', '/'):
                op = tok[1]
                self.consume('OP')
                right = self.unary()
                node = ('binop', op, node, right)
            else:
                break
        return node

    def unary(self):
        tok = self.peek()
        if tok[0] == 'OP' and tok[1] in ('+', '-', '++', '--'):
            op = tok[1]
            self.consume('OP')
            node = self.unary()
            return ('unop', op, node)
        else:
            return self.primary()

    def primary(self):
        tok = self.peek()
        if tok[0] == 'NUMBER':
            self.consume('NUMBER')
            return ('num', tok[1])
        elif tok[0] == 'ID':
            id_name = tok[1]
            self.consume('ID')
            # function call
            if self.peek()[0] == 'LBRACS' and self.peek()[1] == '(':
                self.consume('LBRACS', '(')
                args = []
                if not (self.peek()[0] == 'RBRACS' and self.peek()[1] == ')'):
                    while True:
                        args.append(self.expression())
                        if self.peek()[0] == 'PUNC' and self.peek()[1] == ',':
                            self.consume('PUNC', ',')
                        else:
                            break
                self.consume('RBRACS', ')')
                return ('call', id_name, args)
            # array access
            elif self.peek()[0] == 'LBRACS' and self.peek()[1] == '[':
                self.consume('LBRACS', '[')
                index_expr = self.expression()
                self.consume('RBRACS', ']')
                return ('array_access', id_name, index_expr)
            else:
                return ('id', id_name)
        elif tok[0] == 'LBRACS' and tok[1] == '(':
            self.consume('LBRACS', '(')
            node = self.expression()
            self.consume('RBRACS', ')')
            return node
        else:
            raise SyntaxError(f"Unexpected token {tok} at line {tok[2]}, col {tok[3]}")


In [15]:
tokens = list(lexer(code))
parser = Parser(tokens)
ast = parser.parse()

import pprint
pprint.pprint(ast)


('program',
 [('for',
   ('for_init_decls', [('declaration', 'int', 'i', ('num', 0))]),
   ('binop', '<', ('id', 'i'), ('num', 100)),
   ('assign', ('id', 'i'), ('binop', '+', ('id', 'i'), ('num', 1))),
   ('block',
    [('expr_stmt',
      ('assign',
       ('array_access', 'a', ('id', 'i')),
       ('binop',
        '+',
        ('array_access', 'a', ('binop', '+', ('id', 'i'), ('num', 1))),
        ('num', 1))))]))])
